In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity ,euclidean_distances
import time



In [2]:
movies = pd.read_csv(r'C:\Users\rajes\Google Drive\Masters\CSE6242 - Data and Visual Analytics\Projects\movie_rec_filtered.csv',lineterminator='\n')



# movies = movies1.iloc[:4000,:]
# movies1.shape

In [3]:
movies.shape

(58499, 27)

In [4]:
movies.info()
movies.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58499 entries, 0 to 58498
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   0                  58499 non-null  int64  
 1   movieId            58499 non-null  int64  
 2   title              58499 non-null  object 
 3   genres             58499 non-null  object 
 4   genres_tmdb        57857 non-null  object 
 5   imdbId             58499 non-null  int64  
 6   tmdbId             58499 non-null  int64  
 7   budget             58499 non-null  int64  
 8   homepage           10521 non-null  object 
 9   id                 58499 non-null  int64  
 10  original_language  58499 non-null  object 
 11  original_title     58499 non-null  object 
 12  overview           58350 non-null  object 
 13  popularity         58499 non-null  float64
 14  release_date       58493 non-null  object 
 15  revenue            58499 non-null  int64  
 16  runtime            584

0                        0
movieId                  0
title                    0
genres                   0
genres_tmdb            642
imdbId                   0
tmdbId                   0
budget                   0
homepage             47978
id                       0
original_language        0
original_title           0
overview               149
popularity               0
release_date             6
revenue                  0
runtime                  0
status                   0
tagline              29059
title_tmdb               0
vote_average             0
vote_count               0
adult                    0
keywords             14895
directors                0
actors                   0
reviews              49185
dtype: int64

In [5]:
movies.head()

,0,movieId,title,genres,genres_tmdb,imdbId,tmdbId,budget,homepage,id,...,status,tagline,title_tmdb,vote_average,vote_count,adult,keywords,directors,actors,reviews
0,0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Animation|Adventure|Family|Comedy|,114709,862,30000000,http://toystory.disney.com/toy-story,862,...,Released,NaN,Toy Story,7.966,15937,False,martial arts|jealousy|friendship|bullying|elem...,John Lasseter|,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,This movie came out when I was three. Now I'm ...
1,1,2,Jumanji (1995),Adventure|Children|Fantasy,Adventure|Fantasy|Family|,113497,8844,65000000,http://www.sonypictures.com/movies/jumanji/,8844,...,Released,Roll the dice and unleash the excitement!,Jumanji,7.234,9258,False,giant insect|board game|jungle|disappearance|r...,Joe Johnston|,Robin Williams|Kirsten Dunst|Bradley Pierce|Bo...,"Throw the dice and take a turn, Jumanji made t..."
2,2,3,Grumpier Old Men (1995),Comedy|Romance,Romance|Comedy|,113228,15602,25000000,NaN,15602,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.432,307,False,fishing|halloween|sequel|old man|best friend|w...,Howard Deutch|,Walter Matthau|Jack Lemmon|Ann-Margret|Sophia ...,NaN
3,3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Comedy|Drama|Romance|,114885,31357,16000000,NaN,31357,...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.300,128,False,based on novel or book|interracial relationshi...,Forest Whitaker|,Whitney Houston|Angela Bassett|Loretta Devine|...,NaN
4,4,5,Father of the Bride Part II (1995),Comedy,Comedy|Family|,113041,11862,0,NaN,11862,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,6.222,605,False,parent child relationship|baby|midlife crisis|...,Charles Shyer|,Steve Martin|Diane Keaton|Martin Short|Kimberl...,NaN


In [6]:
# Returns the list top 3 elements or entire list
def get_list(x):
    x = str(x).split("|")
    if isinstance (x, list):
       # names = [i for i in x]
        
        if len(x) > 5:
            x =  x[:5]
        return x
    #Return empty list in case there is no data
    return []

In [7]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace( " ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''


In [8]:
def movie_recommender(title_tmdb, cosine_sim):
    if title_tmdb in movies_list['title_tmdb'].values :
        idx = indices[title_tmdb]
        sim_scores= list(enumerate(cosine_sim[idx]))
        sim_scores =sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]
        
        movie_indices = [i[0] for i in sim_scores]
        
        print( "Here is the list of Top 10 movies, similar to Title \"" + title_tmdb + "\"")
        similar_movies = movies_list['title_tmdb'].iloc[movie_indices].to_frame()
        
        return similar_movies.style.hide_index()
        
    else:
        print ("Movie \"" + title_tmdb + "\" in not present in our database, please search for another movie")

In [9]:
def movie_recommender1(title_tmdb, cosine_sim):
    if title_tmdb in movies_list['title_tmdb'].values :
        idx = indices[title_tmdb]

        sim_scores= list(enumerate(cosine_sim[idx]))

        sim_scores =sorted(sim_scores, key=lambda x: x[1], reverse=True)

        sim_scores_as_df = pd.DataFrame(sim_scores, columns=['index','sim_score'])
        movies_final = movies_list.merge(sim_scores_as_df, how='left', left_on='index_col', right_on='index')
        movies_final = movies_final.sort_values('sim_score', ascending=False)
        similar_movies = movies_final[['title_tmdb', 'sim_score']].head(11).tail(10)
        print( "Here is the list of Top 10 movies, similar to Title \"" + title_tmdb + "\"")
        return similar_movies.style.hide_index()

    else:
        print ("Movie \"" + title_tmdb + "\" in not present in our database, please search for another movie")

In [10]:
# Calcualting cosine_similarity in batches
def cosine_similarity_n_space(m1, m2, batch_size=50):
    assert m1.shape[1] == m2.shape[1]
    ret = np.ndarray((m1.shape[0], m2.shape[0]))
    for row_i in range(0, int(m1.shape[0] / batch_size) + 1):
        start = row_i * batch_size
        end = min([(row_i + 1) * batch_size, m1.shape[0]])
        if end <= start:
            break # cause I'm too lazy to elegantly handle edge cases
        rows = m1[start: end]
        sim = cosine_similarity(rows, m2) # rows is O(1) size
        ret[start: end] = sim
    return ret


In [11]:
# Get the Top 3 elements from cast, genres and Keywords attribute
features = ['actors', 'keywords' , 'genres', 'directors']
for feature in features:
    movies[feature] = movies[feature].apply(get_list)

In [12]:
C=movies['vote_average'].astype(float).mean()
C

5.923657703550483

In [13]:
features = ['actors', 'keywords' , 'genres', 'directors']

for feature in features:
    movies[feature] =  movies[feature].apply(clean_data)

In [14]:
movies.head()

,0,movieId,title,genres,genres_tmdb,imdbId,tmdbId,budget,homepage,id,...,status,tagline,title_tmdb,vote_average,vote_count,adult,keywords,directors,actors,reviews
0,0,1,Toy Story (1995),"[adventure, animation, children, comedy, fantasy]",Animation|Adventure|Family|Comedy|,114709,862,30000000,http://toystory.disney.com/toy-story,862,...,Released,NaN,Toy Story,7.966,15937,False,"[martialarts, jealousy, friendship, bullying, ...","[johnlasseter, ]","[tomhanks, timallen, donrickles, jimvarney, wa...",This movie came out when I was three. Now I'm ...
1,1,2,Jumanji (1995),"[adventure, children, fantasy]",Adventure|Fantasy|Family|,113497,8844,65000000,http://www.sonypictures.com/movies/jumanji/,8844,...,Released,Roll the dice and unleash the excitement!,Jumanji,7.234,9258,False,"[giantinsect, boardgame, jungle, disappearance...","[joejohnston, ]","[robinwilliams, kirstendunst, bradleypierce, b...","Throw the dice and take a turn, Jumanji made t..."
2,2,3,Grumpier Old Men (1995),"[comedy, romance]",Romance|Comedy|,113228,15602,25000000,NaN,15602,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.432,307,False,"[fishing, halloween, sequel, oldman, bestfriend]","[howarddeutch, ]","[waltermatthau, jacklemmon, ann-margret, sophi...",NaN
3,3,4,Waiting to Exhale (1995),"[comedy, drama, romance]",Comedy|Drama|Romance|,114885,31357,16000000,NaN,31357,...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.300,128,False,"[basedonnovelorbook, interracialrelationship, ...","[forestwhitaker, ]","[whitneyhouston, angelabassett, lorettadevine,...",NaN
4,4,5,Father of the Bride Part II (1995),[comedy],Comedy|Family|,113041,11862,0,NaN,11862,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,6.222,605,False,"[parentchildrelationship, baby, midlifecrisis,...","[charlesshyer, ]","[stevemartin, dianekeaton, martinshort, kimber...",NaN


In [15]:


def create_combine_overview(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['actors']) +  ' ' + ' '  .join(x['directors']) + ' ' + str(x['overview']) +  ' ' + ' '.join(x['genres'] )


In [16]:
# quartile 90 for getting which  got votes more than 90% 
m=movies['vote_count'].quantile(0.9)
m
m =1

In [17]:

# Replacing NaN with empty string
movies['overview'] = movies['overview'].fillna('')
movies['combine_overview'] =movies.apply(create_combine_overview, axis=1)

movies[['title_tmdb', 'actors', 'directors', 'genres', 'keywords', 'combine_overview']].head()


,title_tmdb,actors,directors,genres,keywords,combine_overview
0,Toy Story,"[tomhanks, timallen, donrickles, jimvarney, wa...","[johnlasseter, ]","[adventure, animation, children, comedy, fantasy]","[martialarts, jealousy, friendship, bullying, ...",martialarts jealousy friendship bullying eleme...
1,Jumanji,"[robinwilliams, kirstendunst, bradleypierce, b...","[joejohnston, ]","[adventure, children, fantasy]","[giantinsect, boardgame, jungle, disappearance...",giantinsect boardgame jungle disappearance rec...
2,Grumpier Old Men,"[waltermatthau, jacklemmon, ann-margret, sophi...","[howarddeutch, ]","[comedy, romance]","[fishing, halloween, sequel, oldman, bestfriend]",fishing halloween sequel oldman bestfriend wal...
3,Waiting to Exhale,"[whitneyhouston, angelabassett, lorettadevine,...","[forestwhitaker, ]","[comedy, drama, romance]","[basedonnovelorbook, interracialrelationship, ...",basedonnovelorbook interracialrelationship sin...
4,Father of the Bride Part II,"[stevemartin, dianekeaton, martinshort, kimber...","[charlesshyer, ]",[comedy],"[parentchildrelationship, baby, midlifecrisis,...",parentchildrelationship baby midlifecrisis con...


In [18]:

movies_list =  movies.copy().loc[movies['vote_count']>=m]


In [19]:
movies_list.shape

(57463, 28)

In [20]:
def weighted_rating(x, m=m, C=C):
    v=x['vote_count']
    R=x['vote_average']
    # calculation done using  IMDB formula
    return (v/(v+m) * R) + (m/(m+v) *C)

In [21]:
movies_list['score']= movies_list.apply(weighted_rating, axis=1)

In [22]:
# movies_list =  movies.copy()

In [23]:
movies_list = movies_list.sort_values('score', ascending=False)
movies_list[['title_tmdb', 'vote_count', 'vote_average','score','popularity']].head(10)

,title_tmdb,vote_count,vote_average,score,popularity
57301,American Rescue Squad,8,10.000,9.547073,1.778
55503,Home Run Showdown,29,9.600,9.477455,3.558
29845,Profile of a Killer,11,9.600,9.293638,2.992
57496,When the Mountains Tremble,5,9.600,8.987276,1.400
51418,National Theatre Live: The Curious Incident of...,3,10.000,8.980914,3.281
57716,Mon Amour Mon Parapluie,3,10.000,8.980914,0.931
53156,The Social Life of Small Urban Spaces,3,10.000,8.980914,0.600
41457,Crazy Horse,6,9.417,8.917951,1.845
56181,Dominion,50,8.960,8.900464,6.192
56470,The Phantom 52,6,9.300,8.817665,1.320


In [24]:
movies_list.shape

(57463, 29)

In [25]:
movies_list = movies_list.reset_index()
indices = pd.Series(movies_list.index, index=movies_list['title_tmdb']).drop_duplicates()

movies_list.head()

,index,0,movieId,title,genres,genres_tmdb,imdbId,tmdbId,budget,homepage,...,title_tmdb,vote_average,vote_count,adult,keywords,directors,actors,reviews,combine_overview,score
0,57301,60235,204848,American Rescue Squad (2015),"[action, comedy]",Comedy|Action|Drama|Mystery|Fantasy|,2261983,340183,0,NaN,...,American Rescue Squad,10.0,8,False,"[musical, superhero, ]","[elliotdiviney, ]","[tonyd.czech, douglassidney, rogerwayne, sarah...",NaN,musical superhero tonyd.czech douglassidney r...,9.547073
1,55503,58268,200026,Home Run Showdown (2012),[children],Family|,1680311,128450,0,NaN,...,Home Run Showdown,9.6,29,False,[nan],"[ozscott, ]","[deancain, matthewlillard, annabethgish, barry...",NaN,nan deancain matthewlillard annabethgish barry...,9.477455
2,29845,30642,137465,Profile Of A Killer (2013),[thriller],Thriller|,2387543,204767,0,NaN,...,Profile of a Killer,9.6,11,False,"[murder, serialkiller, ]","[caspiantredwell-owen, ]","[joeypollari, gabrieleangieri, emilyfradenburg...",NaN,murder serialkiller joeypollari gabrieleangie...,9.293638
3,57496,60454,205455,When the Mountains Tremble (1983),[documentary],Documentary|History|War|,421089,259882,0,NaN,...,When the Mountains Tremble,9.6,5,False,"[cia, guerrillawarfare, centralandsouthamerica...","[newtonthomassigel, pamelayates, ]","[rigobertamenchú, ]",NaN,cia guerrillawarfare centralandsouthamerica in...,8.987276
4,51418,53802,189845,National Theatre Live: The Curious Incident of...,[(nogenreslisted)],Drama|,3786680,272076,0,NaN,...,National Theatre Live: The Curious Incident of...,10.0,3,False,"[womandirector, ]","[nickwickham, ]","[luketreadaway, niamhcusack, paulritter, nicol...",NaN,womandirector luketreadaway niamhcusack paulr...,8.980914


In [26]:
movies_list['index_col'] = movies_list.index

In [27]:
#  Defining a TF-IDF vectorizer object and Removing all stop words
tfidf = TfidfVectorizer(stop_words='english')



# Making the TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(movies_list['combine_overview'])

tfidf_matrix

<57463x204848 sparse matrix of type '<class 'numpy.float64'>'
	with 2039248 stored elements in Compressed Sparse Row format>

In [28]:
tfidf_matrix.shape

(57463, 204848)

In [29]:


# cosine_sim  =cosine_similarity(tfidf_matrix, tfidf_matrix)
# print(cosine_sim)

# print("Time taken: %s seconds" %(time.time() - start))

In [30]:
# # Calculating the Cosine Similarity matrix
start = time.time()
cosine_sim =cosine_similarity_n_space(tfidf_matrix, tfidf_matrix, batch_size=100)
print("Time taken: %s seconds" %(time.time() - start))

Time taken: 151.03317046165466 seconds


In [31]:
# # Calculating the Cosine Similarity matrix
# start = time.time()

# cosine_sim1  =linear_kernel(tfidf_matrix, tfidf_matrix)
# print(cosine_sim1)

# print("Time taken: %s seconds" %(time.time() - start))

In [32]:
# count = CountVectorizer(stop_words='english')
# count_matrix = count.fit_transform(movies['combine_overview'])

# count_matrix.shape
# count_matrix.A

In [33]:
# start = time.time()

# cosine_sim2  = cosine_similarity(count_matrix, count_matrix)
# print(cosine_sim2)

# print("Time taken: %s seconds" %(time.time() - start))

In [34]:
movies_list.head()

,index,0,movieId,title,genres,genres_tmdb,imdbId,tmdbId,budget,homepage,...,vote_average,vote_count,adult,keywords,directors,actors,reviews,combine_overview,score,index_col
0,57301,60235,204848,American Rescue Squad (2015),"[action, comedy]",Comedy|Action|Drama|Mystery|Fantasy|,2261983,340183,0,NaN,...,10.0,8,False,"[musical, superhero, ]","[elliotdiviney, ]","[tonyd.czech, douglassidney, rogerwayne, sarah...",NaN,musical superhero tonyd.czech douglassidney r...,9.547073,0
1,55503,58268,200026,Home Run Showdown (2012),[children],Family|,1680311,128450,0,NaN,...,9.6,29,False,[nan],"[ozscott, ]","[deancain, matthewlillard, annabethgish, barry...",NaN,nan deancain matthewlillard annabethgish barry...,9.477455,1
2,29845,30642,137465,Profile Of A Killer (2013),[thriller],Thriller|,2387543,204767,0,NaN,...,9.6,11,False,"[murder, serialkiller, ]","[caspiantredwell-owen, ]","[joeypollari, gabrieleangieri, emilyfradenburg...",NaN,murder serialkiller joeypollari gabrieleangie...,9.293638,2
3,57496,60454,205455,When the Mountains Tremble (1983),[documentary],Documentary|History|War|,421089,259882,0,NaN,...,9.6,5,False,"[cia, guerrillawarfare, centralandsouthamerica...","[newtonthomassigel, pamelayates, ]","[rigobertamenchú, ]",NaN,cia guerrillawarfare centralandsouthamerica in...,8.987276,3
4,51418,53802,189845,National Theatre Live: The Curious Incident of...,[(nogenreslisted)],Drama|,3786680,272076,0,NaN,...,10.0,3,False,"[womandirector, ]","[nickwickham, ]","[luketreadaway, niamhcusack, paulritter, nicol...",NaN,womandirector luketreadaway niamhcusack paulr...,8.980914,4


In [35]:
cosine_sim.shape

(57463, 57463)

In [36]:
movie_recommender('Toy Story', cosine_sim=cosine_sim)

Here is the list of Top 10 movies, similar to Title "Toy Story"


title_tmdb
Toy Story 3
Toy Story 2
Buzz Lightyear of Star Command: The Adventure Begins
The Dizzy Acrobat
Small Fry
Toy Story 4
The 40 Year Old Virgin
Andy Hardy's Blonde Trouble
Hawaiian Vacation
Andy Kaufman Plays Carnegie Hall


In [37]:
movie_recommender1('Toy Story', cosine_sim=cosine_sim)

Here is the list of Top 10 movies, similar to Title "Toy Story"


title_tmdb,sim_score
Toy Story 3,0.499365
Toy Story 2,0.473686
Buzz Lightyear of Star Command: The Adventure Begins,0.297157
The Dizzy Acrobat,0.294085
Small Fry,0.288833
Toy Story 4,0.284797
The 40 Year Old Virgin,0.218664
Andy Hardy's Blonde Trouble,0.196936
Hawaiian Vacation,0.194759
Andy Kaufman Plays Carnegie Hall,0.188857


In [40]:
movie_keywords = {'title_tmdb': ['Toy Story','Jumanji'] }

df = pd.DataFrame(movie_keywords, columns = ['title_tmdb'])

In [41]:
df

,title_tmdb
0,Toy Story
1,Jumanji


In [42]:
from IPython.display import display
for index, row in df.iterrows():
    globals()[f"movie_{index}"]= movie_recommender1(row['title_tmdb'],cosine_sim)

display (movie_0)
display (movie_1)

Here is the list of Top 10 movies, similar to Title "Toy Story"
Here is the list of Top 10 movies, similar to Title "Jumanji"


title_tmdb,sim_score
Toy Story 3,0.499365
Toy Story 2,0.473686
Buzz Lightyear of Star Command: The Adventure Begins,0.297157
The Dizzy Acrobat,0.294085
Small Fry,0.288833
Toy Story 4,0.284797
The 40 Year Old Virgin,0.218664
Andy Hardy's Blonde Trouble,0.196936
Hawaiian Vacation,0.194759
Andy Kaufman Plays Carnegie Hall,0.188857


title_tmdb,sim_score
Table No. 21,0.158764
Liar Game: Reborn,0.151673
The Slender Thread,0.144916
Quintet,0.143714
Scooby-Doo! and the Cyber Chase,0.141000
Jumanji: Welcome to the Jungle,0.139990
The Dark Angel,0.139931
Game Box 1.0,0.139491
Snowed Under,0.135781
The Surrounding Game,0.132725
